In [1]:
import win32com.client as win32
import os
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Define the location to save your files
output_folder_path = 'Your_Output_Folder'  # replace with the path of your folder

# Define the subject string to look for and the number of days
subject_string = "Your_Subject_String"  # replace with your subject string
num_days = 30  # replace with your number of days

# Start Outlook
Outlook = win32.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = Outlook.GetDefaultFolder(6)  # 6 refers to the inbox

# Get all emails
messages = inbox.Items

# Get the current date and time
now = datetime.now(pytz.utc)

# Define the Excel file path
excel_file_path = os.path.join(output_folder_path, 'Output_Filename.xlsx')

# Create an Excel writer object
writer = pd.ExcelWriter(excel_file_path, engine='openpyxl')

# Loop through all emails in reverse order (most recent first)
for message in reversed(messages):

    # If the subject contains the specified subject string and the email was received within the specified number of days
    if subject_string in message.Subject and now - message.ReceivedTime < timedelta(days=num_days):

        # Write the HTML body to a temporary file
        temp_file_path = os.path.join(output_folder_path, 'temp.html')
        with open(temp_file_path, 'w', encoding='utf-8') as file:
            file.write(message.HTMLBody)

        # Attempt to read the first table from the HTML file into a pandas DataFrame
        try:
            table = pd.read_html(temp_file_path)[0]
        except ValueError:
            print("No tables found in this email. Skipping...")
            continue

        df = table  # assuming the table you want is the first one

        # Remove columns that are identical to their neighboring column
        if df.shape[1] > 0:  # if df has at least one column
            cols_to_drop = []  # start with an empty list
            for i in range(df.shape[1]-1):  # iterate through all columns, except the last one
                if df.iloc[:, i].equals(df.iloc[:, i+1]):  # if this column is equal to the next one
                    cols_to_drop.append(df.columns[i+1])  # add the next column to the list

            # Now drop all the columns in the list
            df.drop(cols_to_drop, axis=1, inplace=True)

        # Create a timestamp from the received time of the email
        timestamp = datetime.strftime(message.ReceivedTime, '%Y-%m-%d')

        # Write the table to a sheet in the Excel file
        df.to_excel(writer, sheet_name=f'{timestamp}', index=False)

        # Delete the temporary HTML file
        os.remove(temp_file_path)

# Save the Excel file
writer.save()


No tables found in this email. Skipping...
No tables found in this email. Skipping...


C:\Users\User\AppData\Local\Temp\ipykernel_21664\1865219229.py:63: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
